In [1]:
import numpy as np
import pandas as pd
import requests
import json
import time
import polyline
import folium
from folium.plugins import BeautifyIcon
import math
import osmnx as ox

from geopy.geocoders import Nominatim
import geopy.distance

/opt/anaconda3/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
with open('strava_tokens.json') as file:
    tokens = json.load(file)

In [3]:
if tokens['expires_at'] < time.time():
    print('Access token expired.')
    response = requests.post(
                        url = 'https://www.strava.com/oauth/token',
                        data = {
                                'client_id': <CLIENT_ID>,
                                'client_secret': <CLIENT_SECRET>,
                                'grant_type': 'refresh_token',
                                'refresh_token': tokens['refresh_token']
                                }
                    )
    tokens = response.json()
    with open('strava_tokens.json', 'w') as outfile:
        json.dump(tokens, outfile)
else:
    print('Access token still valid.')
    
access_token = tokens['access_token']

Access token still valid.


In [4]:
def lines_and_markers(points, start_or_end, names, f_map, show_poly=True):
    
    if show_poly:
        folium.PolyLine(points, color='red').add_to(f_map)
    
    lats = []
    longs = []

    for i,j in points:
        lats.append(i)
        longs.append(j)

    mid_index = math.floor(len(lats)/2)
    mark_points = [lats[mid_index], longs[mid_index]]
#     folium.CircleMarker(mark_points, radius=2, tooltip=names).add_to(f_map)
    icon_cust = BeautifyIcon(
        icon_shape='doughnut', 
        icon_size=[11,11],
        border_color='red', 
        border_width=2,
    )
    
#     folium.Marker(mark_points, tooltip=names, icon=icon_cust).add_to(f_map)
    folium.Marker(start_or_end, tooltip=names, icon=icon_cust).add_to(f_map)
    
    return points

In [5]:
class QuadTree():
    def __init__(self, lat, long, mi, df_list, poly_list, depth, max_depth):
        self.lat = lat
        self.long = long
        self.miles = mi
        self.center = (lat, long)
        self.df_list = df_list
        self.poly_list = poly_list
        self.depth = depth
        self.max_depth = max_depth

    def subdivide(self):
        
        if len(self.poly_list) >= self.max_depth:
            return None
        
        else:
            # Calculate directions using center    
            north = geopy.distance.distance(self.miles).destination((self.lat, self.long), bearing=0)[:2]
            east = geopy.distance.distance(self.miles).destination((self.lat, self.long), bearing=90)[:2]
            south= geopy.distance.distance(self.miles).destination((self.lat, self.long), bearing=180)[:2]
            west = geopy.distance.distance(self.miles).destination((self.lat, self.long), bearing=270)[:2]

            # Query box
            url = f'https://www.strava.com/api/v3/segments/explore?bounds={south[0]}, {west[1]}, {north[0]}, {east[1]}&activity_type=riding'
            header = {'Authorization': 'Bearer ' + access_token}
            top_seg = requests.get(url, headers=header)
            print(top_seg.reason)
            top_seg = top_seg.json()
            try: 
                self.df_list.append(pd.DataFrame(top_seg['segments']))
            except KeyError:
                return None
    #         self.seg_df = self.seg_df.append(pd.DataFrame(top_seg['segments']), ignore_index=True)

            # Save box lines for plotting
            northwest = (north[0], west[1])
            northeast = (north[0], east[1])
            southeast = (south[0], east[1])
            southwest = (south[0], west[1])
            outer_box_poly = [northwest, northeast, southeast, southwest, northwest]
            self.poly_list.append(outer_box_poly)

        
            if len(top_seg['segments']) == 10 or len(self.poly_list) == 1:

                # Calculate nw center & subdivide
                nw_lat = (north[0] + west[0])/2
                nw_long = (north[1] + west[1])/2
    #             print('in nw')
                nw = QuadTree(nw_lat, nw_long, self.miles/2, self.df_list, self.poly_list, self.depth + 1, self.max_depth)
                nw.subdivide()
    #             print('subdivided nw')

                # Calculate ne center & subdivide
                ne_lat = (north[0] + east[0])/2
                ne_long = (north[1] + east[1])/2
    #             print('in ne')
                ne = QuadTree(ne_lat, ne_long, self.miles/2, self.df_list, self.poly_list, self.depth + 1, self.max_depth)
                ne.subdivide()
    #             print('subdivided ne')

                # Calculate se center & subdivide
                se_lat = (south[0] + east[0])/2
                se_long = (south[1] + east[1])/2
    #             print('in se')
                se = QuadTree(se_lat, se_long, self.miles/2, self.df_list, self.poly_list, self.depth + 1, self.max_depth)
                se.subdivide()
    #             print('subdivided se')

                # Calculate sw center & subdivide
                sw_lat = (south[0] + west[0])/2
                sw_long = (south[1] + west[1])/2
    #             print('in sw')
                sw = QuadTree(sw_lat, sw_long, self.miles/2, self.df_list, self.poly_list, self.depth + 1, self.max_depth)
                sw.subdivide()
    #             print('subdivided sw')
    

In [6]:
metro = 'Jackson, TN'
geolocator = Nominatim(user_agent="drews_app")
location = geolocator.geocode(metro)

In [8]:
city = ox.geocode_to_gdf(metro)
city_bb = pd.DataFrame(city).iloc[:, 1:5]
city_bb = dict(zip(city_bb.columns, [i for i in city_bb.values[0]]))

city_poly = [
    (city_bb['bbox_north'], city_bb['bbox_west']),
    (city_bb['bbox_north'], city_bb['bbox_east']),
    (city_bb['bbox_south'], city_bb['bbox_east']),
    (city_bb['bbox_south'], city_bb['bbox_west']),
    (city_bb['bbox_north'], city_bb['bbox_west'])
]  

In [9]:
qt = QuadTree(location.latitude, location.longitude, 5, [], [], 0, 100)
qt.subdivide()

OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK


In [10]:
len(qt.poly_list)

13

In [11]:
# qt.poly_list

In [12]:
qt_df = pd.concat(qt.df_list, ignore_index=True)
# print(qt_df['id'].duplicated().sum())
qt_df = qt_df.drop_duplicates(subset='id')
print(qt_df.shape)
qt_df['Decoded Polyline'] = qt_df['points'].apply(polyline.decode)

(22, 14)


In [13]:
qt_df.to_csv(f'{metro}_segments.csv')

In [14]:
#  tile = 'OpenStreetMap'
# tile = 'Stamen Terrain'
# tile = 'Stamen Toner'
# tile = 'Stamen Watercolor'
# tile = 'CartoDB positron'
tile = 'CartoDB dark_matter'

mi = 3
center = (location.latitude, location.longitude)
north = geopy.distance.distance(miles=mi).destination(center, bearing=0)[:2]
east = geopy.distance.distance(miles=mi).destination(center, bearing=90)[:2]
south= geopy.distance.distance(miles=mi).destination(center, bearing=180)[:2]
west = geopy.distance.distance(miles=mi).destination(center, bearing=270)[:2]

z = folium.Map(tiles=tile)
z.fit_bounds([[south[0], west[1]], [north[0], east[1]]])
qt_df['Decoded Polyline'] = qt_df.apply(lambda x: lines_and_markers(x['Decoded Polyline'], x['end_latlng'], x['name'], f_map=z, show_poly=False), axis=1)
folium.PolyLine(qt.poly_list, color='blue').add_to(z)
z.save(f'{metro}_segPoint.html')
z

In [15]:
#  tile = 'OpenStreetMap'
# tile = 'Stamen Terrain'
# tile = 'Stamen Toner'
# tile = 'Stamen Watercolor'
# tile = 'CartoDB positron'
tile = 'CartoDB dark_matter'

mi = 10
center = (location.latitude, location.longitude)
north = geopy.distance.distance(miles=mi).destination(center, bearing=0)[:2]
east = geopy.distance.distance(miles=mi).destination(center, bearing=90)[:2]
south= geopy.distance.distance(miles=mi).destination(center, bearing=180)[:2]
west = geopy.distance.distance(miles=mi).destination(center, bearing=270)[:2]

z = folium.Map(tiles=tile)
z.fit_bounds([[south[0], west[1]], [north[0], east[1]]])
qt_df['Decoded Polyline'] = qt_df.apply(lambda x: lines_and_markers(x['Decoded Polyline'], x['end_latlng'], x['name'], f_map=z, show_poly=True), axis=1)
folium.PolyLine(qt.poly_list, color='blue').add_to(z)
folium.PolyLine(city_poly, color='green').add_to(z)
z.save(f'{metro}_segPoly.html')
z

In [16]:
one_mile = 1609.34
qt_df = qt_df[(qt_df['distance'] <= one_mile) & (qt_df['distance'] >= one_mile/10)]
qt_df.shape

(19, 15)